<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [575]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [576]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [577]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [578]:
# текст запроса
query_3_1 = f''' 
select
    count(*) as N_row,
    count(distinct id) as N_ids,
    count(distinct name) as N_names
from
    vacancies
limit 10
'''

In [579]:
vacancies_df = pd.read_sql_query(query_3_1, connection)
vacancies_df

,n_row,n_ids,n_names
0,49197,49197,21223


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [580]:
query_3_2 = '''
select
    count(distinct name) as N_employers,
    count(distinct id) as N_ids,
    count(*) as N_rows
from
    employers
'''

In [581]:
employers_df = pd.read_sql_query(query_3_2, connection)
employers_df

,n_employers,n_ids,n_rows
0,23175,23501,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [582]:
query_3_3 = ''' 
select
    count(distinct name) as N_areas,
    count(distinct id) as N_ids
from
    areas   
'''

In [583]:
areas_df = pd.read_sql_query(query_3_3, connection)
areas_df

,n_areas,n_ids
0,1362,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [584]:
query_3_4 =''' 
select
    count(distinct name) as N_areas,
    count(distinct id) as N_ids
from
    industries       
'''

In [585]:
idustries_df = pd.read_sql_query(query_3_4, connection)
idustries_df

,n_areas,n_ids
0,294,294


***

1. В таблице vacancies, которая отражает общее количество вакансий, 49197 строк. <br>
Кличество уникальных названий вакансий составило 21223
2. В таблице employers, которая отражает общее количество работодателей, 23501 строка. <br>
Кличество уникальных названий работадателя составило 23175
3. В таблице areas, которая отражает общее количество регионов, 1362 строки. <br>
Все названия (names) уникальны.

4. В таблице industries которая отражает количество сфер деятельности, 294 строки <br>
Все названия (names) уникальны.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [586]:
query_4_1 = ''' 
select
    a.name,
    count(*) N_vacs
from
    vacancies v
    join areas a
    on v.area_id = a.id
group by a.name
order by count(*) desc
limit 10
'''

In [587]:
VacAreaJoin = pd.read_sql_query(query_4_1, connection)
VacAreaJoin

,name,n_vacs
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [588]:
query_4_2 = ''' 
select
    count(*) as N_slry_non_null
from 
    vacancies
where 
    (salary_from is not null) or (salary_to is not null)
'''


In [589]:
slry_not_null = pd.read_sql_query(query_4_2, connection)
slry_not_null

,n_slry_non_null
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [590]:
query_4_3 = ''' 
select
    round(avg(salary_from)) as avg_slry_from,
    round(avg(salary_to)) as avg_slry_to
from 
    vacancies

'''

In [591]:
avg_slry = pd.read_sql_query(query_4_3, connection)
avg_slry

,avg_slry_from,avg_slry_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. <br> Результат отсортируйте по убыванию количества.


In [592]:
query_4_4 = ''' 
select
    schedule,
    employment,
    count(*) N_vac
from
    vacancies
group by 
    schedule, employment
order by count(*) desc
'''

In [593]:
sch_empl_count = pd.read_sql_query(query_4_4, connection)
display(sum(sch_empl_count.loc[2:, 'n_vac']))
sch_empl_count

6028

,schedule,employment,n_vac
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [594]:
query_4_5 = ''' 
select
    experience,
    count(*) as N_vac
from
    vacancies
group by experience
order by count(*) asc
'''

In [595]:
count_by_exp = pd.read_sql_query(query_4_5, connection)
count_by_exp

,experience,n_vac
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

1. Наибольшее количество вакансий представлено в Москве и Санкт-Петербурге. Также, в топе столицы сосдних государств и города-миллионники.

2. Только у 24073 соискателей из 49197 (чуть менее половины) указана хотя бы одна граница зарплатной вилки

3. В среднем, нижняя граница зарплатной вилки составила 71к рублей, верхней - 110к рублей

4. Соискатели, преимущественно, предпочитают полную занятость и полный день (~35к). На втором месте удаленная работа и полная занятость (~8к).<br>
На все остальные сочетания графика работы и занятости приходится ~6к
5. Подавляющее бльшинство соискателей (~26к) имеет опыть работы до 3-х лет. От 3-х до 6-ти лет у ~14к соискателей. <br>
~7к соискателей не имеют опыта работы. Опыт работы более 6 лет имеют всего 1337 соискателей.

ЗАМЕНИТЬ СОИСКАТЕЛЕЙ НА ВАКАНСИИ


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [596]:
query_5_1 = ''' 
select
    e.name,
    count(*) N_vacs
from
    vacancies v
    join employers e
    on v.employer_id = e.id
group by
    e.name  
order by count(*) desc
'''

In [597]:
vac_count_by_empl = pd.read_sql_query(query_5_1, connection)
vac_count_by_empl.iloc[:5, :]

,name,n_vacs
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём. <br>
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [598]:
query_5_2 =''' 
select
    a.name,
    count(e.name) N_employers
    
from
    areas a
    left join vacancies v 
    on v.area_id = a.id
    
    join employers e 
    on a.id = e.area
    
where 
    v.id is null
    
group by 
    a.name

order by 
    count(e.name) desc
'''

In [599]:
Null_vac = pd.read_sql_query(query_5_2, connection)
Null_vac

,name,n_employers
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Беларусь,18
...,...,...
181,Миллерово,1
182,"Михайловка (Уфимский район, Башкортостан)",1
183,Можайск,1
184,Морозовск\n,1


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [600]:
query_5_3 = ''' 
select
    e.name as Employer_name,
    count(distinct a.name) as N_regions_with_vac
from
    vacancies v
    join employers e
    on v.employer_id = e.id
    
    join areas a
    on v.area_id = a.id
    
group by 
    e.name
    
order by 
    count(distinct a.name) desc
'''


In [601]:
N_region_by_Employer = pd.read_sql_query(query_5_3, connection)
N_region_by_Employer

,employer_name,n_regions_with_vac
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [602]:
query_5_4 = ''' 
select 
    count(e.name) N_empl_with_no_indstr
from
    employers e
    left join employers_industries ei 
    on ei.employer_id = e.id
where industry_id is null

'''


In [603]:
N_Empl_null_Indstr = pd.read_sql_query(query_5_4, connection)
N_Empl_null_Indstr

,n_empl_with_no_indstr
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [604]:
query_5_5 = ''' 
select 
    e.name,
    count(ei.industry_id)
    
from
    employers_industries ei
    join employers e
    on ei.employer_id = e.id 
    
group by 
    e.name

having 
    count(ei.industry_id) = 4
    
order by 
    e.name
    
offset 2
limit 1
'''



In [605]:
Indstr_by_Empl = pd.read_sql_query(query_5_5, connection)
Indstr_by_Empl

,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [606]:
query_5_6 = ''' 
select 
    i.name,
    count (*) N_emplyers
    
from
    employers_industries ei
    join employers e
    on ei.employer_id = e.id 
    
    join industries i
    on ei.industry_id = i.id

group by 
    i.name
    
having 
    i.name = 'Разработка программного обеспечения'
'''


In [607]:
N_dev_Empl = pd.read_sql_query(query_5_6, connection)
N_dev_Empl

,name,n_emplyers
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [608]:
import requests
from bs4 import BeautifulSoup

# Устанавливаем библиотеки BeautifulSoup и lxml

#!pip install beautifulsoup4 
#!pip install lxml

# Выделяем таблицу с названиями городов-миллионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_="standard sortable")


# Создаем список, в котором элементами являются столбцы таблицы
# Записываем в список названия городов 
df = pd.read_html(str(table))
df = pd.concat(df) 
city_names = list(df.loc[:,'Город'])
display(city_names)


['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Красноярск',
 'Нижний Новгород',
 'Челябинск',
 'Уфа',
 'Самара',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. <br>
Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

In [658]:
query_5_7 = ''' 
select 
    a.name as City,
    v.id as vacancy_cnt
    
from 
    vacancies v
    join employers e
    on v.employer_id = e.id
    
    join areas a
    on v.area_id = a.id

where e.name = 'Яндекс' 

'''


In [659]:
# Записываем результат запроса в датафрейм
YaVac_by_City = pd.read_sql_query(query_5_7, connection)


# Выбираем только данные из городов-миллионников
mask = YaVac_by_City.loc[:, 'city'].apply(lambda x: x in city_names)
YaVac_by_City = YaVac_by_City[mask]


# Группируем по переменной 'City', подссчитываем кол-во вакансий
YaVac_by_City = \
    YaVac_by_City.groupby('city').count()\
        .sort_values('vacancy_cnt', ascending=False)


# Добавляем строрку 'Total' с суммой по солбцу с вакансиями
YaVac_by_City.loc['Total'] = sum(YaVac_by_City['vacancy_cnt'])
display(YaVac_by_City)


,vacancy_cnt
city,
Москва,54
Санкт-Петербург,42
Екатеринбург,39
Нижний Новгород,36
Новосибирск,35
Воронеж,32
Краснодар,30
Самара,26
Уфа,26


In [611]:
# результат запроса

***

In [612]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [613]:
# текст запроса

In [614]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [615]:
# текст запроса

In [616]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [617]:
# текст запроса

In [618]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [619]:
# текст запроса

In [620]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [621]:
# текст запроса

In [622]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [623]:
# текст запроса

In [624]:
# результат запроса

***

In [625]:
# выводы по предметному анализу

# Общий вывод по проекту

In [626]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования